<a href="https://colab.research.google.com/github/AryanAthreya/CSprojects/blob/main/Contextual%20Chatbot%20-%20NLP%20and%20Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# Libraries needed for Keras processing
import numpy as np
from tensorflow import keras
import random
import json
import pickle

# Build the model using Keras instead of tflearn
def create_model(input_size, output_size):
    model = keras.Sequential([
        keras.layers.Dense(10, input_shape=(input_size,), activation='relu'),
        keras.layers.Dense(10, activation='relu'),
        keras.layers.Dense(output_size, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Data preprocessing functions remain the same
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return np.array(bag)

# Training data preparation
def prepare_training_data(intents):
    words = []
    classes = []
    documents = []
    ignore = ['?']

    for intent in intents['intents']:
        for pattern in intent['patterns']:
            w = nltk.word_tokenize(pattern)
            words.extend(w)
            documents.append((w, intent['tag']))
            if intent['tag'] not in classes:
                classes.append(intent['tag'])

    words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
    words = sorted(list(set(words)))
    classes = sorted(list(set(classes)))

    training = []
    output_empty = [0] * len(classes)

    for doc in documents:
        bag = []
        pattern_words = doc[0]
        pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
        for w in words:
            bag.append(1) if w in pattern_words else bag.append(0)

        output_row = list(output_empty)
        output_row[classes.index(doc[1])] = 1
        training.append([bag, output_row])

    random.shuffle(training)
    training = np.array(training, dtype=object)

    train_x = list(training[:,0])
    train_y = list(training[:,1])

    return words, classes, train_x, train_y

# Classification and response functions
ERROR_THRESHOLD = 0.25
def classify(sentence, model, words, classes):
    results = model.predict(np.array([bow(sentence, words)]))[0]
    results = [[i,r] for i,r in enumerate(results) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def response(sentence, model, words, classes, intents, userID='123', show_details=False):
    context = {}
    results = classify(sentence, model, words, classes)

    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    if 'context_set' in i:
                        if show_details:
                            print('context:', i['context_set'])
                        context[userID] = i['context_set']

                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details:
                            print('tag:', i['tag'])
                        return random.choice(i['responses'])
            results.pop(0)
    return "I'm not sure how to respond to that."

# Main training and setup code
def train_chatbot(intents_file):
    with open(intents_file) as json_data:
        intents = json.load(json_data)

    words, classes, train_x, train_y = prepare_training_data(intents)

    model = create_model(len(train_x[0]), len(train_y[0]))

    # Convert training data to numpy arrays
    train_x = np.array(train_x)
    train_y = np.array(train_y)

    # Train the model
    model.fit(
        train_x, train_y,
        epochs=1000,
        batch_size=8,
        verbose=1
    )

    # Save training data
    training_data = {
        'words': words,
        'classes': classes,
        'train_x': train_x.tolist(),
        'train_y': train_y.tolist()
    }
    with open('training_data', 'wb') as f:
        pickle.dump(training_data, f)

    # Save the model
    model.save('chatbot_model.h5')

    return model, words, classes, intents

model, words, classes, intents = train_chatbot('content.json')



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.0170 - loss: 3.7390
Epoch 2/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.0434 - loss: 3.7129
Epoch 3/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1208 - loss: 3.6731
Epoch 4/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1086 - loss: 3.6283
Epoch 5/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1322 - loss: 3.5391
Epoch 6/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1622 - loss: 3.4336
Epoch 7/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2221 - loss: 3.2497
Epoch 8/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2435 - loss: 3.0660
Epoch 9/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2729 - loss: 2.8685
Epoch 10/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3161 - loss: 2.6469
Epoch 11/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3681 - loss: 2.4842
Epoch 12/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
We're open every day 9am-9pm


##ask the model

In [33]:
# Use the chatbot
user_input = "choose a name"
print(response(user_input, model, words, classes, intents))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
I am your helper


In [40]:
from google.colab import files

# Download the saved model to your local machine
files.download("chatbot_model.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
!pip install nltk
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [36]:
# Instead of:
# model.save('chatbot_model.h5')

# Use this:
model.save('chatbot_model.keras')

In [38]:
from IPython.display import FileLink

display(FileLink('chatbot_model.keras'))

/content/chatbot_model.keras

In [1]:
%%writefile content.json
{"intents": [
        {"tag": "greeting",
         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day"],
         "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help?"],
         "context_set": ""
        },
        {"tag": "goodbye",
         "patterns": ["Bye", "See you later", "Goodbye"],
         "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]
        },
        {"tag": "thanks",
         "patterns": ["Thanks", "Thank you", "That's helpful"],
         "responses": ["Happy to help!", "Any time!", "My pleasure"]
        },
        {"tag": "hours",
         "patterns": ["What hours are you open?", "What are your hours?", "When are you open?" ],
         "responses": ["We're open every day 9am-9pm", "Our hours are 9am-9pm every day"]
        },
        {"tag": "location",
         "patterns": ["What is your location?", "Where are you located?", "What is your address?", "Where is your restaurant situated?" ],
         "responses": ["We are on the intersection of London Alley and Bridge Avenue.", "We are situated at the intersection of London Alley and Bridge Avenue", "Our Address is: 1000 Bridge Avenue, London EC3N 4AJ, UK"]
        },
        {"tag": "payments",
         "patterns": ["Do you take credit cards?", "Do you accept Mastercard?", "Are you cash only?" ],
         "responses": ["We accept VISA, Mastercard and AMEX", "We accept most major credit cards"]
        },
        {"tag": "todaysmenu",
         "patterns": ["What is your menu for today?", "What are you serving today?", "What is today's special?"],
         "responses": ["Today's special is Chicken Tikka", "Our speciality for today is Chicken Tikka"]
        },
        {"tag": "deliveryoption",
         "patterns": ["Do you provide home delivery?", "Do you deliver the food?", "What are the home delivery options?" ],
         "responses": ["Yes, we provide home delivery through UBER Eats and Zomato?", "We have home delivery options through UBER Eats and Zomato"],
         "context_set": "food"
        },
        {"tag": "menu",
         "patterns": ["What is your Menu?", "What are the main course options?", "Can you tell me the most delicious dish from the menu?", "What is the today's special?"],
         "responses": ["You can visit www.mymenu.com for menu options", "You can check out the food menu at www.mymenu.com", "You can check various delicacies given in the food menu at www.mymenu.com"],
         "context_filter": "food"
        },
        {
                "tag": "greeting",
                "patterns": [
                   "Hi",
                   "How are you?",
                   "Is anyone there?",
                   "Hello",
                   "Good day",
                   "What's up",
                   "how are ya",
                   "heyy",
                   "whatsup",
                   "??? ??? ??"
                ],
                "responses": [
                   "Hello!",
                   "Good to see you again!",
                   "Hi there, how can I help?"
                ],
                "context_set": ""
             },
              {
                "tag": "goodbye",
                "patterns": [
                   "cya",
                   "see you",
                   "bye bye",
                   "See you later",
                   "Goodbye",
                   "I am Leaving",
                   "Bye",
                   "Have a Good day",
                   "talk to you later",
                   "ttyl",
                   "i got to go",
                   "gtg"
                ],
                "responses": [
                   "Sad to see you go :(",
                   "Talk to you later",
                   "Goodbye!",
                   "Come back soon"
                ],
                "context_set": ""
             },
              {
                "tag": "creator",
                "patterns": [
                   "what is the name of your developers",
                   "what is the name of your creators",
                   "what is the name of the developers",
                   "what is the name of the creators",
                   "who created you",
                   "your developers",
                   "your creators",
                   "who are your developers",
                   "developers",
                   "you are made by",
                   "you are made by whom",
                   "who created you",
                   "who create you",
                   "creators",
                   "who made you",
                   "who designed you"
                ],
                "responses": [
                   "College students"
                ],
                "context_set": ""
             },
              {
                "tag": "name",
                "patterns": [
                   "name",
                   "your name",
                   "do you have a name",
                   "what are you called",
                   "what is your name",
                   "what should I call you",
                   "whats your name?",
                   "what are you",
                   "who are you",
                   "who is this",
                   "what am i chatting to",
                   "who am i taking to",
                   "what are you"
                ],
                "responses": [
                   "You can call me Mind Reader.",
                   "I'm Mind Reader",
                   "I am a Chatbot.",
                   "I am your helper"
                ],
                "context_set": ""
             },
              {
                "tag": "hours",
                "patterns": [
                   "timing of college",
                   "what is college timing",
                   "working days",
                   "when are you guys open",
                   "what are your hours",
                   "hours of operation",
                   "when is the college open",
                   "college timing",
                   "what about college timing",
                   "is college open on saturday",
                   "tell something about college timing",
                   "what is the college  hours",
                   "when should i come to college",
                   "when should i attend college",
                   "what is my college time",
                   "college timing",
                   "timing college"
                ],
                "responses": [
                   "College is open 8am-5pm Monday-Saturday!"
                ],
                "context_set": ""
             },
              {
                "tag": "number",
                "patterns": [
                   "more info",
                   "contact info",
                   "how to contact college",
                   "college telephone number",
                   "college number",
                   "What is your contact no",
                   "Contact number?",
                   "how to call you",
                   "College phone no?",
                   "how can i contact you",
                   "Can i get your phone number",
                   "how can i call you",
                   "phone number",
                   "phone no",
                   "call"
                ],
                "responses": [
                   "You can contact at: NUMBER"
                ],
                "context_set": ""
             },
              {
                "tag": "course",
                "patterns": [
                   "list of courses",
                   "list of courses offered",
                   "list of courses offered in",
                   "what are the courses offered in your college?",
                   "courses?",
                   "courses offered",
                   "courses offered in (your univrsity(UNI) name)",
                   "courses you offer",
                   "branches?",
                   "courses available at UNI?",
                   "branches available at your college?",
                   "what are the courses in UNI?",
                   "what are branches in UNI?",
                   "what are courses in UNI?",
                   "branches available in UNI?",
                   "can you tell me the courses available in UNI?",
                   "can you tell me the branches available in UNI?",
                   "computer engineering?",
                   "computer",
                   "Computer engineering?",
                   "it",
                   "IT",
                   "Information Technology",
                   "AI/Ml",
                   "Mechanical engineering",
                   "Chemical engineering",
                   "Civil engineering"
                ],
                "responses": [
                   "Our university offers Information Technology, computer Engineering, Mechanical engineering,Chemical engineering, Civil engineering and extc Engineering."
                ],
                "context_set": ""
             },
              {
                "tag": "fees",
                "patterns": [
                   "information about fee",
                   "information on fee",
                   "tell me the fee",
                   "college fee",
                   "fee per semester",
                   "what is the fee of each semester",
                   "what is the fees of each year",
                   "what is fee",
                   "what is the fees",
                   "how much is the fees",
                   "fees for first year",
                   "fees",
                   "about the fees",
                   "tell me something about the fees",
                   "What is the fees of hostel",
                   "how much is the fees",
                   "hostel fees",
                   "fees for AC room",
                   "fees for non-AC room",
                   "fees for Ac room for girls",
                   "fees for non-Ac room for girls",
                   "fees for Ac room for boys",
                   "fees for non-Ac room for boys"
                ],
                "responses": [
                   "For Fee detail visit <a target=\"_blank\" href=\"LINK\"> here</a>"
                ],
                "context_set": ""
             },
              {
                "tag": "location",
                "patterns": [
                   "where is the college located",
                   "college is located at",
                   "where is college",
                   "where is college located",
                   "address of college",
                   "how to reach college",
                   "college location",
                   "college address",
                   "wheres the college",
                   "how can I reach college",
                   "whats is the college address",
                   "what is the address of college",
                   "address",
                   "location"
                ],
                "responses": [
                   "<a target=\"_blank\" href=\"ADD YOU GOOGLE MAP LINK HERE\"> here</a>"
                ],
                "context_set": ""
             },
              {
                "tag": "hostel",
                "patterns": [
                   "hostel facility",
                   "hostel servive",
                   "hostel location",
                   "hostel address",
                   "hostel facilities",
                   "hostel fees",
                   "Does college provide hostel",
                   "Is there any hostel",
                   "Where is hostel",
                   "do you have hostel",
                   "do you guys have hostel",
                   "hostel",
                   "hostel capacity",
                   "what is the hostel fee",
                   "how to get in hostel",
                   "what is the hostel address",
                   "how far is hostel from college",
                   "hostel college distance",
                   "where is the hostel",
                   "how big is the hostel",
                   "distance between college and hostel",
                   "distance between hostel and college"
                ],
                "responses": [
                   "For hostel detail visit <a target=\"_blank\" href=\"ADD YOUR HOSTEL DETAIL PDF LINK OR ANY INFORMATION LINK OR ADD YOU OWN ANSWERS\"> here</a>"
                ],
                "context_set": ""
             },
              {
                "tag": "event",
                "patterns": [
                   "events organised",
                   "list of events",
                   "list of events organised in college",
                   "list of events conducted in college",
                   "What events are conducted in college",
                   "Are there any event held at college",
                   "Events?",
                   "functions",
                   "what are the events",
                   "tell me about events",
                   "what about events"
                ],
                "responses": [
                   "For event detail visit <a target=\"_blank\" href=\"ADD YOUR FUNCTIONS LINK OR YOUR OWN RESPONSE\"> here</a>"
                ],
                "context_set": ""
             },
              {
                "tag": "document",
                "patterns": [
                   "document to bring",
                   "documents needed for admision",
                   "documents needed at the time of admission",
                   "documents needed during admission",
                   "documents required for admision",
                   "documents required at the time of admission",
                   "documents required during admission",
                   "What document are required for admission",
                   "Which document to bring for admission",
                   "documents",
                   "what documents do i need",
                   "what documents do I need for admission",
                   "documents needed"
                ],
                "responses": [
                   "To know more about document required visit <a target=\"_blank\" href=\"ADD LINK OF ADMISSION GUIDANCE DOCUMENT FROM YOUR UNIVERSITY WEBSITE\"> here</a>"
                ],
                "context_set": ""
             },
              {
                "tag": "floors",
                "patterns": [
                   "size of campus",
                   "building size",
                   "How many floors does college have",
                   "floors in college",
                   "floors in college",
                   "how tall is UNI's College of Engineering college building",
                   "floors"
                ],
                "responses": [
                   "My College has total 2 floors "
                ],
                "context_set": ""
             },
              {
                "tag": "syllabus",
                "patterns": [
                   "Syllabus for IT",
                   "what is the Information Technology syllabus",
                   "syllabus",
                   "timetable",
                   "what is IT syllabus",
                   "syllabus",
                   "What is next lecture"
                ],
                "responses": [
                   "Timetable provide direct to the students OR To know about syllabus visit <a target=\"_blank\" href=\"TIMETABLE LINK\"> here</a>"
                ],
                "context_set": ""
             },
              {
                "tag": "library",
                "patterns": [
                   "is there any library",
                   "library facility",
                   "library facilities",
                   "do you have library",
                   "does the college have library facility",
                   "college library",
                   "where can i get books",
                   "book facility",
                   "Where is library",
                   "Library",
                   "Library information",
                   "Library books information",
                   "Tell me about library",
                   "how many libraries"
                ],
                "responses": [
                   "There is one huge and spacious library.timings are 8am to 6pm and for more visit <a target=\"blank\" href=\"ADD LIBRARY DETAIL LINK\">here</a>"
                ],
                "context_set": ""
             },
              {
                "tag": "infrastructure",
                "patterns": [
                   "how is college infrastructure",
                   "infrastructure",
                   "college infrastructure"
                ],
                "responses": [
                   "Our University has Excellent Infrastructure. Campus is clean. Good IT Labs With Good Speed of Internet connection"
                ],
                "context_set": ""
             },
              {
                "tag": "canteen",
                "patterns": [
                   "food facilities",
                   "canteen facilities",
                   "canteen facility",
                   "is there any canteen",
                   "Is there a cafetaria in college",
                   "Does college have canteen",
                   "Where is canteen",
                   "where is cafetaria",
                   "canteen",
                   "Food",
                   "Cafetaria"
                ],
                "responses": [
                   "Our university has canteen with variety of food available"
                ],
                "context_set": ""
             },
              {
                "tag": "menu",
                "patterns": [
                   "food menu",
                   "food in canteen",
                   "Whats there on menu",
                   "what is available in college canteen",
                   "what foods can we get in college canteen",
                   "food variety",
                   "What is there to eat?"
                ],
                "responses": [
                   "we serve Franky, Locho, Alu-puri, Kachori, Khavsa, Thaali and many more on menu"
                ],
                "context_set": ""
             },
              {
                "tag": "placement",
                "patterns": [
                   "What is college placement",
                   "Which companies visit in college",
                   "What is average package",
                   "companies visit",
                   "package",
                   "About placement",
                   "placement",
                   "recruitment",
                   "companies"
                ],
                "responses": [
                   "To know about placement visit <a target=\"_blank\" href=\"PLACEMENT INFORMATION LINK FROM YOUR UNIVERSITY WEBSITE IF THEY HAVE\">here</a>"
                ],
                "context_set": ""
             },
              {
                "tag": "ithod",
                "patterns": [
                   "Who is HOD",
                   "Where is HOD",
                   "it hod",
                   "name of it hod"
                ],
                "responses": [
                   "All engineering departments have only one hod XYZ who available on (Place name)"
                ],
                "context_set": ""
             },
              {
                "tag": "computerhod",
                "patterns": [
                   "Who is computer HOD",
                   "Where is computer HOD",
                   "computer hod",
                   "name of computer hod"
                ],
                "responses": [
                   "All engineering departments have only one hod XYZ who available on (PLACE NAME)"
                ],
                "context_set": ""
             },
              {
                "tag": "extchod",
                "patterns": [
                   "Who is extc HOD",
                   "Where is  extc HOD",
                   "extc hod",
                   "name of extc hod"
                ],
                "responses": [
                   "Different school wise hod are different.So be more clear with your school or department"
                ],
                "context_set": ""
             },
              {
                "tag": "principal",
                "patterns": [
                   "what is the name of principal",
                   "whatv is the principal name",
                   "principal name",
                   "Who is college principal",
                   "Where is principal's office",
                   "principal",
                   "name of principal"
                ],
                "responses": [
                   "XYZ is college principal and if you need any help then call your branch hod first.That is more appropriate"
                ],
                "context_set": ""
             },
              {
                "tag": "sem",
                "patterns": [
                   "exam dates",
                   "exam schedule",
                   "When is semester exam",
                   "Semester exam timetable",
                   "sem",
                   "semester",
                   "exam",
                   "when is exam",
                   "exam timetable",
                   "exam dates",
                   "when is semester"
                ],
                "responses": [
                   "Here is the Academic Calendar  <a target=\"_blank\" href=\"YOUR ACADEMIC CALENDER\">website</a>"
                ],
                "context_set": ""
             },
              {
                "tag": "admission",
                "patterns": [
                   "what is the process of admission",
                   "what is the admission process",
                   "How to take admission in your college",
                   "What is the process for admission",
                   "admission",
                   "admission process"
                ],
                "responses": [
                   "Application can also be submitted online through the Unversity's  <a target=\"_blank\" href=\"LINK OF ADMISSION DOCUMENT\">website</a>"
                ],
                "context_set": ""
             },
              {
                "tag": "scholarship",
                "patterns": [
                   "scholarship",
                   "Is scholarship available",
                   "scholarship engineering",
                   "scholarship it",
                   "scholarship ce",
                   "scholarship mechanical",
                   "scholarship civil",
                   "scholarship chemical",
                   "scholarship for AI/ML",
                   "available scholarships",
                   "scholarship for computer engineering",
                   "scholarship for IT engineering",
                   "scholarship for mechanical engineering",
                   "scholarship for civil engineering",
                   "scholarship for chemical engineering",
                   "list of scholarship",
                   "comps scholarship",
                   "IT scholarship",
                   "mechanical scholarship",
                   "civil scholarship",
                   "chemical scholarship",
                   "automobile scholarship",
                   "first year scholarship",
                   "second year scholarship",
                   "third year scholarship",
                   "fourth year scholarship"
                ],
                "responses": [
                   "Many government scholarships are supported by our university. For details and updates visit <a target=\"_blank\" href=\"(SCHOLARSHIP DETAILS LINK)\">here</a>"
                ],
                "context_set": ""
             },
              {
                "tag": "facilities",
                "patterns": [
                   "What facilities college provide",
                   "College facility",
                   "What are college facilities",
                   "facilities",
                   "facilities provided"
                ],
                "responses": [
                   "Our university's Engineering department provides fully AC Lab with internet connection, smart classroom, Auditorium, library,canteen"
                ],
                "context_set": ""
             },
              {
                "tag": "college intake",
                "patterns": [
                   "max number of students",
                   "number of seats per branch",
                   "number of seats in each branch",
                   "maximum number of seats",
                   "maximum students intake",
                   "What is college intake",
                   "how many stundent are taken in each branch",
                   "seat allotment",
                   "seats"
                ],
                "responses": [
                   "For IT, Computer and extc 60 per branch and seat may be differ for different department."
                ],
                "context_set": ""
             },
              {
                "tag": "uniform",
                "patterns": [
                   "college dress code",
                   "college dresscode",
                   "what is the uniform",
                   "can we wear casuals",
                   "Does college have an uniform",
                   "Is there any uniform",
                   "uniform",
                   "what about uniform",
                   "do we have to wear uniform"
                ],
                "responses": [
                   "ENTER YOUR OWN UNIVERSITY UNIFORM CIRCULER"
                ],
                "context_set": ""
             },
              {
                "tag": "committee",
                "patterns": [
                   "what are the different committe in college",
                   "different committee in college",
                   "Are there any committee in college",
                   "Give me committee details",
                   "committee",
                   "how many committee are there in college"
                ],
                "responses": [
                   "For the various committe in college contact this number: ADD NUMBER"
                ],
                "context_set": ""
             },
              {
                "tag": "random",
                "patterns": [
                   "I love you",
                   "Will you marry me",
                   "Do you love me"
                ],
                "responses": [
                   "I am not program for this, please ask appropriate query"
                ],
                "context_set": ""
             },
              {
                "tag": "swear",
                "patterns": [
                   "fuck",
                   "bitch",
                   "shut up",
                   "hell",
                   "stupid",
                   "idiot",
                   "dumb ass",
                   "asshole",
                   "fucker"
                ],
                "responses": [
                   "please use appropriate language",
                   "Maintaining decency would be appreciated"
                ],
                "context_set": ""
             },
              {
                "tag": "vacation",
                "patterns": [
                   "holidays",
                   "when will semester starts",
                   "when will semester end",
                   "when is the holidays",
                   "list of holidays",
                   "Holiday in these year",
                   "holiday list",
                   "about vacations",
                   "about holidays",
                   "When is vacation",
                   "When is holidays",
                   "how long will be the vacation"
                ],
                "responses": [
                   "Academic calender is given to you by your class-soordinators after you join your respective classes"
                ],
                "context_set": ""
             },
              {
                "tag": "sports",
                "patterns": [
                   "sports and games",
                   "give sports details",
                   "sports infrastructure",
                   "sports facilities",
                   "information about sports",
                   "Sports activities",
                   "please provide sports and games information"
                ],
                "responses": [
                   "Our university encourages all-round development of students and hence provides sports facilities in the campus. For more details visit<a target=\"_blank\" href=/\"(LINK IF HAVE)\">here</a>"
                ],
                "context_set": ""
             },
              {
                "tag": "salutaion",
                "patterns": [
                   "okk",
                   "okie",
                   "nice work",
                   "well done",
                   "good job",
                   "thanks for the help",
                   "Thank You",
                   "its ok",
                   "Thanks",
                   "Good work",
                   "k",
                   "ok",
                   "okay"
                ],
                "responses": [
                   "I am glad I helped you",
                   "welcome, anything else i can assist you with?"
                ],
                "context_set": ""
             },
              {
                "tag": "task",
                "patterns": [
                   "what can you do",
                   "what are the thing you can do",
                   "things you can do",
                   "what can u do for me",
                   "how u can help me",
                   "why i should use you"
                ],
                "responses": [
                   "I can answer to low-intermediate questions regarding college",
                   "You can ask me questions regarding college, and i will try to answer them"
                ],
                "context_set": ""
             },
              {
                "tag": "ragging",
                "patterns": [
                   "ragging",
                   "is ragging practice active in college",
                   "does college have any antiragging facility",
                   "is there any ragging cases",
                   "is ragging done here",
                   "ragging against",
                   "antiragging facility",
                   "ragging juniors",
                   "ragging history",
                   "ragging incidents"
                ],
                "responses": [
                   "We are Proud to tell you that our college provides ragging free environment, and we have strict rules against ragging"
                ],
                "context_set": ""
             },
              {
                "tag": "hod",
                "patterns": [
                   "hod",
                   "hod name",
                   "who is the hod"
                ],
                "responses": [
                   "HODs differ for each branch, please be more specific like: (HOD it)"
                ],
                "context_set": ""
             }
   ]
}

Writing content.json


# New Section

In [10]:
#importing the dataset
with open('content.json') as content:
  data1 = json.load(content)

In [18]:
# Getting all the data into lists
tags = []
inputs = []
responses = {}

for intent in data1['intents']:
    responses[intent['tag']] = intent['responses']
    for lines in intent['input']:
        inputs.append(lines)
        tags.append(intent['tag'])

KeyError: 'input'

#new02


In [26]:
import numpy as np
import json
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Load dataset
data = {
    "intents": [
        {"tag": "greeting", "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day"], "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help?"], "context_set": ""},
        {"tag": "goodbye", "patterns": ["Bye", "See you later", "Goodbye"], "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]},
        {"tag": "thanks", "patterns": ["Thanks", "Thank you", "That's helpful"], "responses": ["Happy to help!", "Any time!", "My pleasure"]},
        {"tag": "hours", "patterns": ["What hours are you open?", "What are your hours?", "When are you open?"], "responses": ["We're open every day 9am-9pm", "Our hours are 9am-9pm every day"]},
        {"tag": "location", "patterns": ["What is your location?", "Where are you located?", "What is your address?", "Where is your restaurant situated?"], "responses": ["We are on the intersection of London Alley and Bridge Avenue.", "We are situated at the intersection of London Alley and Bridge Avenue", "Our Address is: 1000 Bridge Avenue, London EC3N 4AJ, UK"]},
        {"tag": "payments", "patterns": ["Do you take credit cards?", "Do you accept Mastercard?", "Are you cash only?"], "responses": ["We accept VISA, Mastercard and AMEX", "We accept most major credit cards"]},
        {"tag": "todaysmenu", "patterns": ["What is your menu for today?", "What are you serving today?", "What is today's special?"], "responses": ["Today's special is Chicken Tikka", "Our speciality for today is Chicken Tikka"]},
        {"tag": "deliveryoption", "patterns": ["Do you provide home delivery?", "Do you deliver the food?", "What are the home delivery options?"], "responses": ["Yes, we provide home delivery through UBER Eats and Zomato?", "We have home delivery options through UBER Eats and Zomato"], "context_set": "food"},
        {"tag": "menu", "patterns": ["What is your Menu?", "What are the main course options?", "Can you tell me the most delicious dish from the menu?", "What is the today's special?"], "responses": ["You can visit www.mymenu.com for menu options", "You can check out the food menu at www.mymenu.com", "You can check various delicacies given in the food menu at www.mymenu.com"], "context_filter": "food"}
    ]
}

# Preprocessing
words = []
classes = []
documents = []
ignore_words = ["?", "!", ",", "."]

for intent in data['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
    if intent['tag'] not in classes:
        classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(set(words))

classes = sorted(set(classes))

# Preparing training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in doc[0]]
    for w in words:
        bag.append(1 if w in word_patterns else 0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

# Model building
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Train the model
model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)

# Save the model and data structures
model.save('chatbot_model.h5')
import pickle

with open('words.pkl', 'wb') as words_file:
    pickle.dump(words, words_file)

with open('classes.pkl', 'wb') as classes_file:
    pickle.dump(classes, classes_file)

# Function to predict the response
def chatbot_response(msg):
    bow = [lemmatizer.lemmatize(w.lower()) for w in nltk.word_tokenize(msg)]
    bag = [1 if w in bow else 0 for w in words]
    res = model.predict(np.array([bag]), verbose=0)
    error_threshold = 0.25
    results = [[i, r] for i, r in enumerate(res[0]) if r > error_threshold]
    results.sort(key=lambda x: x[1], reverse=True)

    if results:
        tag = classes[results[0][0]]
        for intent in data['intents']:
            if intent['tag'] == tag:
                return random.choice(intent['responses'])
    return "I didn't understand that."

# Test the chatbot
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    print("Chatbot:", chatbot_response(user_input))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
